In [29]:
import math, glob, time, os, re, json
import numpy as np
import pandas as pd
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt
from tqdm.notebook import trange, tqdm

from katrain.core.base_katrain import KaTrainBase
from katrain.core.game import Game, KaTrainSGF
from katrain.core.engine import KataGoEngine

In [38]:
kt = KaTrainBase(force_package_config=True)
kt.debug_level=1

In [31]:
e_config = kt.config("engine")
e_config["max_visits"] = e_config["fast_visits"] = 1 # super quick - maybe inaccurate
e_config["threads"] = 72
e_config["_include_ownership"] = False
e_config["config"] = "./kata_config.cfg"
e_config["model"] = '../g170-b20c256x2-s4384473088-d968438914.bin.gz'
e_config["model"] = '../g170e-b20c256x2-s5055114240-d1149032340.bin.gz'

e_config

{'katago': '',
 'model': '../g170e-b20c256x2-s5055114240-d1149032340.bin.gz',
 'config': './kata_config.cfg',
 'threads': 72,
 'max_visits': 1,
 'fast_visits': 1,
 'max_time': 3.0,
 'wide_root_noise': 0.0,
 '_enable_ownership': True,
 '_include_ownership': False}

In [32]:
query = {"rules": "chinese", "priority": 0, "analyzeTurns": [361], "maxVisits": 1, "komi": 7.5, "boardXSize": 19, "boardYSize": 19, "moves": [["B", "Q16"], ["W", "D17"], ["B", "D16"], ["W", "C16"], ["B", "C15"], ["W", "Q14"], ["B", "C17"], ["W", "B16"], ["B", "B17"], ["W", "B15"], ["B", "B14"], ["W", "D15"], ["B", "C14"], ["W", "E16"], ["B", "D18"], ["W", "E18"], ["B", "E19"], ["W", "E10"], ["B", "F18"], ["W", "D16"], ["B", "E17"], ["W", "Q17"], ["B", "F16"], ["W", "R17"], ["B", "E15"], ["W", "M3"], ["B", "D14"], ["W", "C4"], ["B", "A15"], ["W", "F4"], ["B", "A16"], ["W", "D7"], ["B", "E11"], ["W", "L6"], ["B", "F10"], ["W", "P4"], ["B", "E9"], ["W", "L15"], ["B", "D10"], ["W", "P13"], ["B", "D8"], ["W", "L10"], ["B", "E7"], ["W", "R8"], ["B", "D6"], ["W", "E5"], ["B", "C7"], ["W", "M8"], ["B", "C5"], ["W", "K18"], ["B", "D5"], ["W", "R4"], ["B", "E6"], ["W", "K12"], ["B", "F5"], ["W", "K16"], ["B", "G4"], ["W", "E2"], ["B", "F3"], ["W", "R16"], ["B", "E4"], ["W", "E3"], ["B", "C3"], ["W", "J2"], ["B", "F2"], ["W", "K14"], ["B", "D3"], ["W", "K5"], ["B", "D2"], ["W", "J4"], ["B", "E1"], ["W", "F12"], ["B", "B4"], ["W", "G16"], ["B", "D4"], ["W", "J8"], ["B", "H4"], ["W", "G17"], ["B", "J5"], ["W", "J9"], ["B", "K6"], ["W", "M4"], ["B", "K4"], ["W", "L5"], ["B", "J3"], ["W", "K2"], ["B", "K3"], ["W", "G13"], ["B", "L2"], ["W", "L3"], ["B", "H2"], ["W", "H6"], ["B", "J1"], ["W", "G9"], ["B", "K1"], ["W", "G18"], ["B", "L4"], ["W", "F11"], ["B", "M5"], ["W", "M6"], ["B", "N5"], ["W", "D12"], ["B", "N4"], ["W", "D11"], ["B", "N3"], ["W", "O6"], ["B", "M2"], ["W", "O5"], ["B", "N6"], ["W", "N7"], ["B", "L7"], ["W", "M7"], ["B", "L8"], ["W", "G7"], ["B", "M9"], ["W", "L11"], ["B", "O7"], ["W", "N8"], ["B", "O8"], ["W", "N9"], ["B", "O9"], ["W", "N10"], ["B", "O10"], ["W", "R2"], ["B", "N11"], ["W", "M10"], ["B", "M11"], ["W", "S9"], ["B", "L9"], ["W", "J6"], ["B", "L12"], ["W", "G10"], ["B", "K11"], ["W", "K10"], ["B", "J11"], ["W", "B10"], ["B", "J10"], ["W", "K7"], ["B", "K9"], ["W", "K8"], ["B", "H7"], ["W", "S4"], ["B", "G8"], ["W", "A10"], ["B", "F7"], ["W", "P2"], ["B", "G6"], ["W", "B12"], ["B", "H5"], ["W", "E12"], ["B", "J7"], ["W", "K6"], ["B", "H8"], ["W", "F14"], ["B", "H9"], ["W", "H10"], ["B", "H11"], ["W", "N1"], ["B", "G11"], ["W", "J12"], ["B", "F9"], ["W", "C13"], ["B", "H12"], ["W", "H13"], ["B", "J13"], ["W", "K13"], ["B", "J14"], ["W", "N2"], ["B", "J15"], ["W", "B8"], ["B", "K15"], ["W", "L14"], ["B", "L16"], ["W", "M13"], ["B", "M15"], ["W", "L17"], ["B", "M14"], ["W", "M16"], ["B", "N13"], ["W", "B7"], ["B", "M12"], ["W", "O12"], ["B", "L13"], ["W", "L16"], ["B", "N16"], ["W", "O16"], ["B", "N17"], ["W", "P16"], ["B", "M17"], ["W", "G15"], ["B", "M18"], ["W", "G19"], ["B", "L18"], ["W", "J18"], ["B", "K19"], ["W", "P7"], ["B", "J19"], ["W", "H16"], ["B", "H18"], ["W", "J16"], ["B", "J17"], ["W", "K17"], ["B", "H17"], ["W", "C10"], ["B", "H19"], ["W", "D13"], ["B", "L19"], ["W", "B6"], ["B", "Q15"], ["W", "P19"], ["B", "P15"], ["W", "O15"], ["B", "O14"], ["W", "N15"], ["B", "P14"], ["W", "R14"], ["B", "N14"], ["W", "S14"], ["B", "P17"], ["W", "O11"], ["B", "O17"], ["W", "F19"], ["B", "F17"], ["W", "C8"], ["B", "F15"], ["W", "E14"], ["B", "B13"], ["W", "A5"], ["B", "A12"], ["W", "P10"], ["B", "B11"], ["W", "M3"], ["B", "C12"], ["W", "P18"], ["B", "C11"], ["W", "C9"], ["B", "A11"], ["W", "M4"], ["B", "B5"], ["W", "A6"], ["B", "A4"], ["W", "O3"], ["B", "C6"], ["W", "O4"], ["B", "D9"], ["W", "M1"], ["B", "A8"], ["W", "A9"], ["B", "G12"], ["W", "L1"], ["B", "F13"], ["W", "E13"], ["B", "H14"], ["W", "R3"], ["B", "G14"], ["W", "H15"], ["B", "K2"], ["W", "A13"], ["B", "L3"], ["W", "B12"], ["B", "P1"], ["W", "N6"], ["B", "O1"], ["W", "O13"], ["B", "Q1"], ["W", "O19"], ["B", "O2"], ["W", "N3"], ["B", "Q2"], ["W", "O18"], ["B", "P3"], ["W", "R15"], ["B", "Q3"], ["W", "R19"], ["B", "Q4"], ["W", "N19"], ["B", "R5"], ["W", "S1"], ["B", "S5"], ["W", "S2"], ["B", "T4"], ["W", "T3"], ["B", "T5"], ["W", "R6"], ["B", "R1"], ["W", "S6"], ["B", "T2"], ["W", "T6"], ["B", "S3"], ["W", "O16"], ["B", "T1"], ["W", "N12"], ["B", "Q6"], ["W", "O15"], ["B", "R7"], ["W", "S7"], ["B", "Q7"], ["W", "A12"], ["B", "T7"], ["W", "S8"], ["B", "Q8"], ["W", "P8"], ["B", "Q9"], ["W", "Q10"], ["B", "R9"], ["W", "R10"], ["B", "S10"], ["W", "S11"], ["B", "T10"], ["W", "T11"], ["B", "T9"], ["W", "T8"], ["B", "P9"], ["W", "P6"], ["B", "Q5"], ["W", "M19"], ["B", "P11"], ["W", "Q12"], ["B", "Q11"], ["W", "A14"], ["B", "R11"], ["W", "S10"], ["B", "S12"], ["W", "R12"], ["B", "T12"], ["W", "J15"], ["B", "S13"], ["W", "P5"], ["B", "R13"], ["W", "E10"], ["B", "Q13"], ["W", "M13"], ["B", "P12"], ["W", "Q12"], ["B", "R12"], ["W", "L12"], ["B", "Q18"], ["W", "E11"], ["B", "Q19"], ["W", "R18"], ["B", "N18"], ["W", "Q19"], ["B", "T14"], ["W", "G14"], ["B", "S15"], ["W", "T15"], ["B", "T16"], ["W", "C12"], ["B", "S16"], ["W", "A17"], ["B", "S17"], ["W", "S18"], ["B", "T18"], ["W", "M12"], ["B", "S19"], ["W", "T19"], ["B", "A18"], ["W", "T17"], ["B", "T15"], ["W", "K9"], ["B", "T18"]], "overrideSettings": {"maxTime": 3.0}, "id": "QUERY:387"}

In [41]:
query['overrideSettings']['nnRandomize'] = False

In [45]:
n=50
results = [None]*n
def set_results(r,i):
    global results
    print(i)
    results[i]=r
    
for i in range(n):
    engine = KataGoEngine(kt, e_config)
    engine.send_query(query,callback=lambda r,ix=i: set_results(r,ix),
                      error_callback=lambda _:_)
    engine.shutdown()
    

Starting KataGo with "C:\Users\sande\Anaconda3\lib\site-packages\katrain\KataGo\katago.exe" analysis -model "C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz" -config "C:\Users\sande\Desktop\katrain-bots\kata_config.cfg" -analysis-threads 72
Sending query QUERY:387: {"rules": "chinese", "priority": 0, "analyzeTurns": [361], "maxVisits": 1, "komi": 7.5, "boardXSize": 19, "boardYSize": 19, "moves": [["B", "Q16"], ["W", "D17"], ["B", "D16"], ["W", "C16"], ["B", "C15"], ["W", "Q14"], ["B", "C17"], ["W", "B16"], ["B", "B17"], ["W", "B15"], ["B", "B14"], ["W", "D15"], ["B", "C14"], ["W", "E16"], ["B", "D18"], ["W", "E18"], ["B", "E19"], ["W", "E10"], ["B", "F18"], ["W", "D16"], ["B", "E17"], ["W", "Q17"], ["B", "F16"], ["W", "R17"], ["B", "E15"], ["W", "M3"], ["B", "D14"], ["W", "C4"], ["B", "A15"], ["W", "F4"], ["B", "A16"], ["W", "D7"], ["B", "E11"], ["W", "L6"], ["B", "F10"], ["W", "P4"], ["B", "E9"], ["W", "L15"], ["B", "D10"], ["W", "P13"], ["B", "D8"], ["W", "L10"]

[2.5][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
0
2020-06-15 19:56:41+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:56:41+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:56:41+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:56:41+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:56:41+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:56:41+0200: Loaded config C:\Users\sande\Desktop\katrain-bots\kata_config.cfg
2020-06-15 19:56:41+0200: Loaded model C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz
2020-06-15 19:56:41+0200: OpenCL backend: Model version 8
2020-06-15 19:56:41+0200: OpenCL backend: Model name: g170-b20c256x2-s5055114240-d114903

2020-06-15 19:56:43+0200: Analysis Engine starting...
2020-06-15 19:56:43+0200: KataGo v1.4.2+kt1.1 (minified)
2020-06-15 19:56:43+0200: nnRandSeed0 = 7922425512177283463
2020-06-15 19:56:43+0200: After dedups: nnModelFile0 = C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz useFP16 auto useNHWC auto
[2.6][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
2
2020-06-15 19:56:44+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:56:44+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:56:44+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:56:44+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:56:44+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:56:44+0200:

2020-06-15 19:56:45+0200: Analysis Engine starting...
2020-06-15 19:56:45+0200: KataGo v1.4.2+kt1.1 (minified)
2020-06-15 19:56:45+0200: nnRandSeed0 = 17934424161746200851
2020-06-15 19:56:45+0200: After dedups: nnModelFile0 = C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz useFP16 auto useNHWC auto
[2.5][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
4
2020-06-15 19:56:47+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:56:47+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:56:47+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:56:47+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:56:47+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:56:47+0200

[2.6][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
6
2020-06-15 19:56:49+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:56:49+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:56:49+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:56:49+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:56:49+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:56:49+0200: Loaded config C:\Users\sande\Desktop\katrain-bots\kata_config.cfg
2020-06-15 19:56:49+0200: Loaded model C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz
2020-06-15 19:56:49+0200: OpenCL backend: Model version 8
2020-06-15 19:56:49+0200: OpenCL backend: Model name: g170-b20c256x2-s5055114240-d114903

...

[2.5][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
40
2020-06-15 19:57:37+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:57:37+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:57:37+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:57:37+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:57:37+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:57:37+0200: Loaded config C:\Users\sande\Desktop\katrain-bots\kata_config.cfg
2020-06-15 19:57:37+0200: Loaded model C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz
2020-06-15 19:57:37+0200: OpenCL backend: Model version 8
2020-06-15 19:57:37+0200: OpenCL backend: Model name: g170-b20c256x2-s5055114240-d11490

[2.6][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
42
2020-06-15 19:57:40+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:57:40+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:57:40+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:57:40+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:57:40+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:57:40+0200: Loaded config C:\Users\sande\Desktop\katrain-bots\kata_config.cfg
2020-06-15 19:57:40+0200: Loaded model C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz
2020-06-15 19:57:40+0200: OpenCL backend: Model version 8
2020-06-15 19:57:40+0200: OpenCL backend: Model name: g170-b20c256x2-s5055114240-d11490

[2.4][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
44
2020-06-15 19:57:43+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:57:43+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:57:43+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:57:43+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:57:43+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:57:43+0200: Loaded config C:\Users\sande\Desktop\katrain-bots\kata_config.cfg
2020-06-15 19:57:43+0200: Loaded model C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz
2020-06-15 19:57:43+0200: OpenCL backend: Model version 8
2020-06-15 19:57:43+0200: OpenCL backend: Model name: g170-b20c256x2-s5055114240-d11490

[2.5][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
46
2020-06-15 19:57:46+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:57:46+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:57:46+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
2020-06-15 19:57:46+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:57:46+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:57:46+0200: Loaded config C:\Users\sande\Desktop\katrain-bots\kata_config.cfg
2020-06-15 19:57:46+0200: Loaded model C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz
2020-06-15 19:57:46+0200: OpenCL backend: Model version 8
2020-06-15 19:57:46+0200: OpenCL backend: Model name: g170-b20c256x2-s5055114240-d11490

2020-06-15 19:57:49+0200: Found OpenCL Platform 0: NVIDIA CUDA (NVIDIA Corporation) (OpenCL 1.2 CUDA 10.2.120)
2020-06-15 19:57:49+0200: Found 1 device(s) on platform 0 with type CPU or GPU or Accelerator
2020-06-15 19:57:49+0200: Found OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) (score 11000102)
[2.7][QUERY:387] KataGo Analysis Received: dict_keys(['id', 'moveInfos', 'rootInfo', 'turnNumber'])
48
2020-06-15 19:57:49+0200: Using OpenCL Device 0: GeForce GTX 1070 (NVIDIA Corporation) OpenCL 1.2 CUDA
2020-06-15 19:57:49+0200: Loaded tuning parameters from: C:\Users\sande/.katrain/opencltuning/tune6_gpuGeForceGTX1070_x19_y19_c256_mv8.txt
2020-06-15 19:57:49+0200: Loaded config C:\Users\sande\Desktop\katrain-bots\kata_config.cfg
2020-06-15 19:57:49+0200: Loaded model C:\Users\sande\Desktop\g170e-b20c256x2-s5055114240-d1149032340.bin.gz
2020-06-15 19:57:49+0200: OpenCL backend: Model version 8
2020-06-15 19:57:49+0200: OpenCL backend: Model name: g170-b20c256x2-s5055114240-d11490

In [46]:
Counter(r['rootInfo']['scoreLead'] for r in results)

Counter({-59.3719367980957: 7,
         -51.47109603881836: 8,
         -57.488765716552734: 6,
         -53.40718460083008: 12,
         -57.42509078979492: 2,
         -54.62522506713867: 2,
         -53.320804595947266: 6,
         -56.813777923583984: 7})

In [30]:
Counter(r['rootInfo']['scoreLead'] for r in results)

Counter({-48.57298278808594: 5,
         -59.3120231628418: 8,
         -48.02152633666992: 8,
         -57.26298141479492: 8,
         -60.98888397216797: 5,
         -59.32369613647461: 6,
         -58.73176193237305: 4,
         -55.52565383911133: 6})